In [1]:
import pandas as pd
import numpy as np

star_df = pd.read_csv('celeb_name_list.csv')        # 연예인 리스트
article_file = open('/Users/hayeong/DataScience/Data-Science/DSlocal/test_data.txt', 'r')      # 크롤링한 기사 제목들의 텍스트 파일 (구분자는 '\n'으로 가정)

# articles_dict: {name: <연예인 이름 리스트>, article: <기사 제목 리스트>}
articles_dict = {'Name':[], 'Article':[]}

star_df.head(10)

,ID,Name,Category,Age
0,2470,BMK,가수,47
1,2469,JK김동욱,가수,44
2,2468,KAI,가수,26
3,2467,MC딩동,배우,41
4,2466,MC몽,가수,41
5,2465,RM,가수,26
6,2464,T.O.P,가수,32
7,2463,가인,가수,33
8,2462,가희,배우,39
9,2461,간미연,가수,38


# articles_dict 생성 후 데이터프레임 만들기
: 기사 제목에 특정 연예인 이름이 들어있으면 'Article', 'Name' 키에 각각 저장 후 article_df 생성

* article: 기사 제목
* star_df: celeb_name_list.csv의 데이터프레임. star_df['Name']에 연예인 이름(=name)이 저장되어있음
* article_df: articles_dict의 데이터프레임. columns=['Name', 'Article'] 


In [2]:
while True:
    # 크롤링한 기사 제목 한 줄을 읽어옴
    article = article_file.readline()
    # 크롤링한 기사 제목을 모두 읽었으면 break
    if article == '':
        break
    article = article[:article.rfind(' ')]       # 기사 제목 뒤의 신문사 이름 제외시키기
    # namelist 안의 모든 연예인 이름에 대해서 기사 제목에 포함되는 이름이 있는 지 확인
    for name in star_df['Name']:
        # 보통 연예인 이름 뒤에 컴마가 오길래 '<이름>,'를 찾는걸로 해봄
        if name+',' in article:
            if not (name in articles_dict['Name']):
                articles_dict['Name'].append(name)
                articles_dict['Article'].append(article+'\n')
            else:
                index = articles_dict['Name'].index(name)
                if not (article in articles_dict['Article'][index]):
                    articles_dict['Article'][index] += article+'\n'
                
        # 제목에 포함된 연예인을 찾았어도 연예인 리스트 전체 탐색함
        # => 기사에 두 명 이상의 연예인이 있을 경우 모두 찾음
article_df = pd.DataFrame(articles_dict)

In [3]:
article_df.head(10)

,Name,Article
0,김혜수,"김혜수, 건강미 넘치는 일상\n"
1,김성령,"김성령, 건강미 넘치는 프리다이빙…""54세 몸매\n"
2,길,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는\n"
3,이하늬,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는\n"
4,김희정,"김희정, 시선 강탈 구릿빛 섹시..건강미에\n"
5,그레이,"'이욱♥'벤, 필라테스 후 건강미 업그레이드..민낯 미모까지\n"
6,레이,"'이욱♥'벤, 필라테스 후 건강미 업그레이드..민낯 미모까지\n'48.6kg' 김지..."
7,지우,"'48.6kg' 김지우, ♥레이먼킴도 놀랄 건강미 가득\n"
8,여진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신\n"
9,진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신\n신현빈, 건강미X고혹미 다 가진 ..."


# count_dict 생성 후 데이터프레임 만들기
: 각각의 연예인 이름이 몇 번 언급되어있는지를 저장함
'Name'키에 연예인 이름, 'Count'키에 언급 횟수를 저장 후 count_df 생성


In [4]:
# count_dict: {Name : 한 번 이상 언급된 연예인 이름 리스트, Count : 언급 횟수}
count_dict = {'Name': articles_dict['Name'], 'Count': []}
i = 0
for name in articles_dict['Name']:
    count_dict['Count'].append(articles_dict['Article'][i].count('\n'))
    i +=1

count_df = pd.DataFrame(count_dict)
count_df.head()

,Name,Count
0,김혜수,1
1,김성령,1
2,길,1
3,이하늬,1
4,김희정,1


# 데이터프레임 전처리

1. 연예인 리스트 데이터프레임(star_df)과 연예인별 언급횟수 데이터프레임(count_df) 합침
2. 언급 되지 않은 연예인이 있는 행은 삭제 (count 컬럼이 NaN인 경우 해당 행 삭제)
3. count 컬럼을 기준으로 내림차순 정렬

In [5]:
# df에 언급 횟수인 Count 컬럼 추가
merged_df = pd.merge(star_df, count_df, on='Name', how = 'outer')
merged_df.head()

,ID,Name,Category,Age,Count
0,2470,BMK,가수,47,NaN
1,2469,JK김동욱,가수,44,NaN
2,2468,KAI,가수,26,NaN
3,2467,MC딩동,배우,41,NaN
4,2466,MC몽,가수,41,NaN


In [6]:
# 언급 되지 않은 연예인이 있는 행은 삭제
temp_df = merged_df.dropna(axis = 0, how = 'any')
# 언급 횟수 기준 내림차순으로 정렬
sorted_df = temp_df.sort_values(by = 'Count', ascending=False,)

# final_df columns: 연예인 정보 + 언급 횟수 + 언급된 기사 제목
final_df = pd.merge(sorted_df, article_df, on='Name', how = 'outer')
final_df.head(20)

,ID,Name,Category,Age,Count,Article
0,1544,비,가수,38,3.0,"에바 포비엘, 건강미 넘치는 브라톱+레깅스 자태..군살이 하나도\n'견미리 딸' 이..."
1,362,진,가수,27,2.0,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신\n신현빈, 건강미X고혹미 다 가진 ..."
2,1840,레이,가수,29,2.0,"'이욱♥'벤, 필라테스 후 건강미 업그레이드..민낯 미모까지\n'48.6kg' 김지..."
3,2400,견미리,배우,55,1.0,"'견미리 딸' 이유비, 구릿빛 피부 공개…건강미\n"
4,1241,엘,가수,28,1.0,"에바 포비엘, 건강미 넘치는 브라톱+레깅스 자태..군살이 하나도\n"
5,81,혁,가수,25,1.0,"‘연애혁명’ 우주소녀 다영, 건강미+걸크러시 가득…스틸컷 공개\n"
6,208,키,가수,29,1.0,"부담스런 건강미.. '호날두 여친' 조지나, 블랙비키니로 시선\n"
7,255,최은주,배우,41,1.0,"최은주, 명품 식스팩 복근…감탄 터지는\n"
8,271,최여진,배우,37,1.0,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신\n"
9,371,지우,배우,22,1.0,"'48.6kg' 김지우, ♥레이먼킴도 놀랄 건강미 가득\n"


# 데이터 확인하기

1. 동명이인인 데이터 확인
2. 상위 20명의 후보 연예인 리스트 확인
3. 후보 연예인들이 언급되었던 기사가 제대로 된 것인지 확인
4. 최종 데이터프레임을 csv 파일로 저장

In [7]:
### 생략 가능 ###
# 동명이인일 시 두 명 다 같은 언급횟수를 가지고 있음  
# ex)김희정(49세), 김희정(28세): 기사 하나에 언급되면 둘 다 count = 1이 됨
# duplicated: 동명이인인 이름의 리스트
duplicated = final_df[final_df.Name.duplicated()]['Name'][:20].tolist()
print('=== 동명이인인 이름 리스트 ===')
print(duplicated)


# 동명이인이 들어간 기사 체크하기
for key in duplicated:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

# 동명이인의 정보 확인하기
duplicated_df = final_df[final_df['Name'].isin(duplicated)]
duplicated_df.head()

=== 동명이인인 이름 리스트 ===
['김희정']
>> 김희정
김희정, 시선 강탈 구릿빛 섹시..건강미에



,ID,Name,Category,Age,Count,Article
20,1922,김희정,배우,28,1.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에\n"
21,1923,김희정,배우,49,1.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에\n"


In [8]:
candidates = final_df['Name'].tolist()[:20]
print('=== Top 20 ===')
print(candidates)

=== Top 20 ===
['비', '진', '레이', '견미리', '엘', '혁', '키', '최은주', '최여진', '지우', '이하늬', '이유비', '유이', '여진', '신현빈', '그레이', '시원', '수영', '류화영', '류시원']


In [9]:
# 상위 20명의 기사가 제대로 된 것인지 체크하기
print('=== Check articles ===')
for key in candidates:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

=== Check articles ===
>> 비
에바 포비엘, 건강미 넘치는 브라톱+레깅스 자태..군살이 하나도
'견미리 딸' 이유비, 구릿빛 피부 공개…건강미
부담스런 건강미.. '호날두 여친' 조지나, 블랙비키니로 시선

>> 진
‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신
신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보'

>> 레이
'이욱♥'벤, 필라테스 후 건강미 업그레이드..민낯 미모까지
'48.6kg' 김지우, ♥레이먼킴도 놀랄 건강미 가득

>> 견미리
'견미리 딸' 이유비, 구릿빛 피부 공개…건강미

>> 엘
에바 포비엘, 건강미 넘치는 브라톱+레깅스 자태..군살이 하나도

>> 혁
‘연애혁명’ 우주소녀 다영, 건강미+걸크러시 가득…스틸컷 공개

>> 키
부담스런 건강미.. '호날두 여친' 조지나, 블랙비키니로 시선

>> 최은주
최은주, 명품 식스팩 복근…감탄 터지는

>> 최여진
‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신

>> 지우
'48.6kg' 김지우, ♥레이먼킴도 놀랄 건강미 가득

>> 이하늬
이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는

>> 이유비
'견미리 딸' 이유비, 구릿빛 피부 공개…건강미

>> 유이
유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는

>> 여진
‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신

>> 신현빈
신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보'

>> 그레이
'이욱♥'벤, 필라테스 후 건강미 업그레이드..민낯 미모까지

>> 시원
류시원, 재혼 후 근황…49세 나이 믿기지 않는

>> 수영
박찬이, 여배우 뺨치는 실내 수영복 화보

>> 류화영
류화영, 건강미 돋보이는 테니스장에서의

>> 류시원
류시원, 재혼 후 근황…49세 나이 믿기지 않는



In [10]:
import qgrid
qgrid_widget = qgrid.show_grid(final_df,show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
# csv로 저장
final_df.to_csv('candidates.csv', index=False, mode='w', encoding='utf-8-sig')
final_df.head(20)